In [ ]:
import scipy.sparse
from sagemaker import image_uris
image_uris.retrieve(framework='tensorflow',region='us-east-1',version='1.12.0',image_scope='inference',instance_type='ml.c5.4xlarge')

In [ ]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 520713654638.dkr.ecr.us-east-1.amazonaws.com

In [ ]:
!docker pull 520713654638.dkr.ecr.us-east-1.amazonaws.com/sagemaker-tensorflow-serving:1.12.0-cpu

In [ ]:
import scipy.sparse
from sagemaker import image_uris
image_uris.retrieve(framework='tensorflow',region='us-east-1',version='2.12.0',image_scope='training',instance_type='ml.c5.4xlarge')

In [ ]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 763104351884.dkr.ecr.us-east-1.amazonaws.com

In [ ]:
!docker build -t tensorflow-training:2.12.0-cpu-py310-vg -f Dockerfile.train .

In [ ]:
!docker images

In [ ]:
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator

# Ruolo SageMaker
role = get_execution_role()

# Parametri del modello
hyperparameters = {'epochs': 10}

# Configurazione dell'estimator
estimator = Estimator(
    image_uri='763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:2.12.0-cpu-py310',
    role=role,
    instance_count=1,
    instance_type='ml.m5.large',
    hyperparameters=hyperparameters,
    output_path='s3://cestino235/output'
)

# Avvio del training
print('##### INIZIO TRAINING #####')
estimator.fit('s3://cestino235/input/vgsales.csv')
print('##### TRAINING COMPLETATO #####')


In [46]:
from sagemaker.tensorflow import TensorFlow
from sagemaker import get_execution_role

role = get_execution_role()

estimator = TensorFlow(
    entry_point='vg.py',  # Nome del file principale
    role=role,
    instance_count=1,
    instance_type='ml.m5.large',
    framework_version='2.12.0',
    py_version='py310',
    output_path='s3://cestino235/output',
    environment={
        'S3_BUCKET_NAME': 'cestino235',
        'S3_FILE_KEY': 'input/vgsales.csv'
    }
)

# Avvio del training
estimator.fit()


[11/22/24 14:10:18] INFO     image_uri is not presented, retrieving image_uri based on            ]8;id=464612;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=391805;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py#674\674]8;;\
                             instance_type, framework etc.                                                         

                    INFO     image_uri is not presented, retrieving image_uri based on            ]8;id=142817;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=217924;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py#674\674]8;;\
                             instance_type, framework etc.                                                         

                    INFO     Creating training-job with name:                                       ]8;id=318435;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=771587;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#1042\1042]8;;\
                             tensorflow-training-2024-11-22-14-10-18-658                                           

2024-11-22 14:10:20 Starting - Starting the training job...
2024-11-22 14:10:34 Starting - Preparing the instances for training...
2024-11-22 14:11:06 Downloading - Downloading input data...
2024-11-22 14:11:46 Downloading - Downloading the training image.........
2024-11-22 14:13:07 Training - Training image download completed. Training in progress.2024-11-22 14:13:12.137635: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX512F, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-22 14:13:14,855 sagemaker-training-toolkit INFO     Imported framework sagemaker_tensorflow_container.training
2024-11-22 14:13:14,856 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-11-22 14:13:14,857 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed

In [ ]:
!tar -xzf data/output/output.tar.gz

In [ ]:
!tar -xzf ./data/output/model.tar.gz

In [ ]:
%%sh
# Nome dell'immagine Docker
image_name="tensorflow-training"
echo "image_name: ${image_name} ######################"

# Account AWS corrente
account=$(aws sts get-caller-identity --query Account --output text)
echo "account: ${account} ######################"

# Regione configurata nel profilo AWS attivo
region=$(aws configure get region)
if [ -z "$region" ]; then
  region="us-west-2"  # Regione di default se non specificata
fi
echo "region: ${region} ######################"

# URI completo dell'immagine Docker su ECR
fullname="${account}.dkr.ecr.${region}.amazonaws.com/${image_name}:2.12.0-cpu-py310-vg"
echo "fullname: ${fullname} ######################"

# Creazione del repository ECR (ignora errore se già esistente)
aws ecr create-repository --repository-name "${image_name}" || echo "Repository ${image_name} già esistente"

# Login a ECR
aws ecr get-login-password --region "${region}" | docker login --username AWS --password-stdin "${account}.dkr.ecr.${region}.amazonaws.com"

# Tag dell'immagine Docker con l'URI completo
docker tag "${image_name}:2.12.0-cpu-py310-vg" "${fullname}"

# Push dell'immagine su ECR
docker push "${fullname}"


In [ ]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 737368952498.dkr.ecr.us-east-1.amazonaws.com

In [ ]:
!docker tag tensorflow-training:2.12.0-cpu-py310-vg 339713156346.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:2.12.0-cpu-py310-vg

In [ ]:
!docker push 339713156346.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:2.12.0-cpu-py310-vg

In [ ]:
!docker ps

In [ ]:
file_path = '{}/vgsales.csv'.format(os.getcwd())
print(os.path.exists(file_path))


In [40]:
role = get_execution_role()
print(role)

arn:aws:iam::339713156346:role/c133245a3375001l7613515t1w33-SageMakerExecutionRole-5pe7px6BaF26


In [ ]:
import os

# Directory corrente
current_dir = os.getcwd()
print(f"Current directory: {current_dir}")

# Verifica del file
file_path = os.path.join(current_dir, "vgsales.csv")
print(f"File path: {file_path}")
print(f"File exists: {os.path.exists(file_path)}")

# Contenuto della directory
print("Contents of the current directory:")
print(os.listdir(current_dir))


In [ ]:
!ls -l /home/ec2-user/SageMaker/AWSrepo/vgsales.csv


In [43]:
import boto3
import os

# Specifica il bucket S3 e il percorso del file
bucket_name = "cestino235"
file_key = "input/vgsales.csv"

# Crea un client S3
s3 = boto3.client('s3')

try:
    # Verifica se il file esiste
    response = s3.head_object(Bucket=bucket_name, Key=file_key)
    print(f"Il file {file_key} esiste nel bucket {bucket_name}.")
    print("Metadati del file:", response)
except s3.exceptions.NoSuchKey:
    print(f"Il file {file_key} non esiste nel bucket {bucket_name}.")
except s3.exceptions.ClientError as e:
    # Errore generico - problemi di permessi o altro
    error_code = e.response['Error']['Code']
    if error_code == '403':
        print(f"Accesso negato al file {file_key}. Verifica i permessi IAM.")
    elif error_code == '404':
        print(f"Il file {file_key} non è stato trovato nel bucket {bucket_name}.")
    else:
        print(f"Errore sconosciuto: {e}")


Il file input/vgsales.csv esiste nel bucket cestino235.
Metadati del file: {'ResponseMetadata': {'RequestId': '2FBFXX7VWH0F3H7S', 'HostId': '2NhTd/I1ByUJEfSmBxhQD/y04CndrOFQTUW7DSlJU/1LW+pcHlBwM7Zkc1dt7RjrcTHy7E1g0Cc=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': '2NhTd/I1ByUJEfSmBxhQD/y04CndrOFQTUW7DSlJU/1LW+pcHlBwM7Zkc1dt7RjrcTHy7E1g0Cc=', 'x-amz-request-id': '2FBFXX7VWH0F3H7S', 'date': 'Fri, 22 Nov 2024 13:48:25 GMT', 'last-modified': 'Fri, 22 Nov 2024 13:13:31 GMT', 'etag': '"67fa6f5efdc100db4586aa80556e8620"', 'x-amz-server-side-encryption': 'AES256', 'accept-ranges': 'bytes', 'content-type': 'text/csv', 'content-length': '1355781', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'AcceptRanges': 'bytes', 'LastModified': datetime.datetime(2024, 11, 22, 13, 13, 31, tzinfo=tzutc()), 'ContentLength': 1355781, 'ETag': '"67fa6f5efdc100db4586aa80556e8620"', 'ContentType': 'text/csv', 'ServerSideEncryption': 'AES256', 'Metadata': {}}


In [47]:
#dopo aver completato il training
!aws s3 cp s3://cestino235/output/tensorflow-training-2024-11-22-14-10-18-658/output/model.tar.gz .


download: s3://cestino235/output/tensorflow-training-2024-11-22-14-10-18-658/output/model.tar.gz to ./model.tar.gz


In [50]:
!tar -xzf model.tar.gz


tar: Ignoring unknown extended header keyword `LIBARCHIVE.creationtime'
